In [ ]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
import logging

def process_table(spark, query_input, output_path):
    try:
        logging.info(f"Starting processing query: {query_input}")
        
        # Reading data from Delta Lake using the query
        df_input_data = spark.sql(query_input)
        
        # Adicionar metadados ou qualquer outra transformação necessária
        df_with_update_date = add_metadata(df_input_data)  # Substitua por sua função de metadados, se necessário
        
        # Remover dados antigos na partição antes de sobrescrever para evitar duplicações
        df_with_update_date.write \
            .format("delta") \
            .option("overwriteSchema", "true") \
            .mode("overwrite") \
            .partitionBy("month_key")  # Ajuste o campo de particionamento conforme necessário
            .save(output_path)
        
        logging.info(f"Query '{query_input}' successfully processed and saved to {output_path}")
    
    except Exception as e:
        logging.error(f"Error processing query '{query_input}': {str(e)}")


def process_table_with_merge(spark, query_input, output_path):
    try:
        logging.info(f"Starting processing query: {query_input}")
        
        # Lendo dados da tabela Delta usando a consulta
        df_input_data = spark.sql(query_input)
        
        # Adicionar metadados ou outras transformações necessárias
        df_with_update_date = add_metadata(df_input_data)  # Substitua por sua função de metadados, se necessário
        
        # Ler a tabela Delta existente
        delta_table = DeltaTable.forPath(spark, output_path)
        
        # Realizar uma operação MERGE para inserir ou atualizar dados sem criar duplicações
        delta_table.alias("old_data") \
            .merge(
                df_with_update_date.alias("new_data"),
                "old_data.id = new_data.id") \
            .whenMatchedUpdate(set={
                "column1": "new_data.column1",
                "column2": "new_data.column2"
                # Adicione mais colunas conforme necessário
            }) \
            .whenNotMatchedInsert(values={
                "id": "new_data.id",
                "column1": "new_data.column1",
                "column2": "new_data.column2"
                # Adicione mais colunas conforme necessário
            }) \
            .execute()
        
        logging.info(f"Query '{query_input}' successfully processed and merged into {output_path}")
    
    except Exception as e:
        logging.error(f"Error processing query '{query_input}': {str(e)}")


def add_metadata(df):
    # Função de exemplo para adicionar metadados
    return df.withColumn("update_date", F.current_timestamp())
